# DZ Сегментация и детекция объектов

## DIGIT RECOGNIZER

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.svm import SVC


In [3]:
# Data 
df_train = pd.read_csv('digit-recognizer/train.csv')
df_test = pd.read_csv('digit-recognizer/test.csv')

# df_train.head()

In [4]:
# df_train.head()
# df_test.shape
df_train.shape

(42000, 785)

In [5]:
# df_train.isnull().any().describe()
df_train.isnull().any().sum()

0

In [6]:
# df_test.isnull().any().describe()
df_test.isnull().any().sum()

0

In [7]:
df_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Separate to data for train-test x & y (data & label)
X_train = df_train.drop(['label'],axis = 1)
Y_train = df_train['label']
X_train.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
Y_train

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [10]:
# Normalization
# According to Kaggle: This pixel-value is an integer between 0 and 255, inclusive.
# So MAX pixel-value is 255
X_train = X_train.astype('float32')
X_train = X_train / 255.0

X_test = df_test.astype('float32')
X_test = df_test / 255.0

X_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Divide train data to train and validation
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2, random_state=42)

In [12]:
# Using PCA
pca = PCA(n_components=100, random_state=42)
X_train_pca =pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca =pca.transform(X_test)

In [13]:
# X_val_pca.shape
# X_train_pca
# Y_train
X_test_pca.shape

(28000, 100)

In [14]:
X_train_pca = pd.DataFrame(X_train_pca)
X_val_pca = pd.DataFrame(X_val_pca)

In [15]:
svc = SVC()
svc.fit(X_train_pca, Y_train)

SVC()

In [16]:
# Using SVM for check train and validation 
y_train_svc = svc.predict(X_train_pca)
y_val_svc = svc.predict(X_val_pca)
svc_train = metrics.accuracy_score(Y_train,y_train_svc)
svc_accuracy = metrics.accuracy_score(Y_val, y_val_svc)

print("Train Accuracy score: {}".format(svc_train))
print("Validation Accuracy score: {}".format(svc_accuracy))
print(metrics.classification_report(Y_val, y_val_svc))

Train Accuracy score: 0.9919345238095238
Validation Accuracy score: 0.9785714285714285
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       816
           1       0.99      0.99      0.99       909
           2       0.98      0.98      0.98       846
           3       0.98      0.97      0.97       937
           4       0.97      0.97      0.97       839
           5       0.98      0.97      0.98       702
           6       0.98      0.99      0.99       785
           7       0.98      0.97      0.98       893
           8       0.97      0.98      0.97       835
           9       0.96      0.97      0.97       838

    accuracy                           0.98      8400
   macro avg       0.98      0.98      0.98      8400
weighted avg       0.98      0.98      0.98      8400



In [17]:
# PREDICTION on test data
y_predict_svc = svc.predict(X_test_pca)
y_predict_svc

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [18]:
y_predict_svc.shape

(28000,)

In [19]:
import csv

In [20]:
# Create file for submitting on Kaggle

fieldnames = ['ImageId','Label']

with open('submit.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fieldnames)
#     csvwriter.writerows(y_predict_svc)
    for i, p in enumerate(y_predict_svc, 1):
        csvwriter.writerow([i,p])